In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=167633'
html = urlopen(url_base + url_sub)
soup = BeautifulSoup(html, 'html.parser')

In [3]:
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=167633&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [4]:
url = url_base + ifr_url
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

In [7]:
count = soup.find('strong', 'total')
count

<strong class="total">관람객 평점 <em>462</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [8]:
count = count.find('em').string   # get_text() 대신에 string
count

'462'

In [9]:
count = int(count.replace(',',''))
count

462

In [10]:
import math
total_page = math.ceil(count/10)
total_page

47

In [11]:
point = []
review = []
date = []
for page in range(1, total_page+1):
    print(page, end=' ')
    url = url_base + ifr_url + '&page=' + str(page)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    result = soup.find(class_='score_result')
    lis = result.find_all('li')
    for li in lis:
        score = li.find(class_='star_score').find('em').get_text()
        score = int(score)
        point.append(score)
        count_span = len(li.find(class_='score_reple').find(class_='').find_all('span'))
        count_icon = len(li.find_all(class_='ico_viewer'))
        count_spo = len(li.find_all(class_='text_spo'))
        criteria = str(count_span) + str(count_icon) + str(count_spo)
        #print(criteria)
        if criteria == '100':
            r = li.find(class_='score_reple').find('span').string.strip(' \t\r\n')
        elif criteria == '200':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '210':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')
        elif criteria == '310':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '311':
            r = li.find(class_='score_reple').find_all('span')[2].string.strip(' \t\r\n')
        elif criteria == '201':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')

        review.append(r)
        date.append(li.find(class_='score_reple').find_all('em')[1].get_text())

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 

In [13]:
import pandas as pd

In [15]:
df = pd.DataFrame({'평점':point, '일시':date, '감상평':review})
df

,평점,일시,감상평
0,8,2019.06.26 21:54,발재간이 압권이었음. ㅋㅋㅋㅋ
1,7,2019.06.26 20:30,흐 지하실 내려가는 거 보고 미쳐 돌아버릴 뻔.. 밀대로 문 부수고 나가든 대가리를...
2,8,2019.06.26 22:49,친구하나 잘 사겼구먼.. 친구말을 잘 들읍시다
3,7,2019.06.27 00:09,"그레타 연기 좋았고 영화 잘 봤지만, 쓰러뜨렸으면 일단 묶어놔 좀..."
4,7,2019.06.26 20:20,아니 클레이 모레츠의 한 주먹꺼리던데 왜 암것두 못하냐구 ㅠ
...,...,...,...
457,4,2019.06.29 00:07,ㅎㅎ 결말 무엇 ㅎㅎ 보지마세욤 그냥 시종일관 고구마 먹는 느낌
458,9,2019.09.07 19:39,클레이 모레츠 재갈물려 묶여있는 모습 좋았어요..잘 쳤습니다.
459,6,2019.06.28 23:30,클로이 답답해서 뒈져버릴 뻔... 초반부터 공감안되긴 함.
460,2,2019.06.29 00:07,하 진짜 제발 보지마세요 최근본영화중에 제일 ㅈ같았고 너무 무리수임...
